In [10]:
import pandas as pd
import numpy as np
import datetime
import psycopg2
import sqlalchemy
import os
from sqlalchemy import create_engine

In [11]:
conn = psycopg2.connect("dbname=cp2 user=postgres password=Ludi24")
cur = conn.cursor()
query_read = "SELECT * FROM stk_enregistrements"
cur.execute(query_read)
raw = cur.fetchall()
conn.commit()

In [12]:
df = pd.DataFrame(raw, columns =["id_enreg","name_volume" ,"capacity" ,"used_capacity","date","client","name_pool" ,"id_typologie" ,"id_origine"  ])
df

,id_enreg,name_volume,capacity,used_capacity,date,client,name_pool,id_typologie,id_origine
0,0,INFS_19DC1_T0M_TEST_EXLIDAT1_01,10.0,9.83,2018-10-03,INF,0,0,0
1,1,INFS_19DC3_T0M_TEST_EXLIDAT2_01,10.0,9.36,2018-10-03,INF,0,0,0
2,2,ELEN_19DC3_T0S_ORAD01_DATA_01,25.0,0.05224609375,2018-10-03,ELE,0,0,0
3,3,ELEN_19DC3_T0S_ORAD01_DATA_02,25.0,0.04443359375,2018-10-03,ELE,0,0,0
4,4,ELEN_19DC3_T0S_ORAD01_DATA_03,25.0,0.03662109375,2018-10-03,ELE,0,0,0
...,...,...,...,...,...,...,...,...,...
681438,681438,IPSS_06T2S_SQL_02,2000.0,1517.4345703125,2021-03-13,IPS,12,4,1
681439,681439,IPSS_06T2S_SQL_02,2000.0,1396.427734375,2021-03-14,IPS,12,4,1
681440,681440,IPSS_06T2S_SQL_02,2000.0,1509.578125,2021-03-15,IPS,12,4,1
681441,681441,IPSS_06T2S_SQL_02,2000.0,1501.6474609375,2021-03-16,IPS,12,4,1


In [54]:
date_bdd = df.date.max()
last_index_id_enreg= df.id_enreg.max()
last_index_id_enreg

681442

In [14]:
basepath_ibm_volume = 'C:/Users/lsarlat/Documents/Capacity_planning/CP3/données_version3/ibm/volume'

fichiers_ibm_volume=[]
with os.scandir(basepath_ibm_volume) as entries: 
    for entry in entries:
        if entry.is_file():
            fichiers_ibm_volume.append(str(entry.name))
print(len(fichiers_ibm_volume))

898


In [15]:
query_read_fichiers = "SELECT * FROM stk_fichiers"
cur.execute(query_read_fichiers)
raw_fichiers = cur.fetchall()
df_fichiers = pd.DataFrame(raw_fichiers, columns =["nom_fichier","origine"])

In [16]:
fichiers_ibm_a_recuperer = []
for i in fichiers_ibm_volume:
    if i not in df_fichiers.nom_fichier.to_list():
        fichiers_ibm_a_recuperer.append(i)
fichiers_ibm_a_recuperer

[]

In [17]:
#pour le test 
fichiers_ibm_a_recuperer = ["20210318_mutsapsan019_lssevdiskcopy.txt"]

In [18]:
import math
def en_MB(x):
    if type(x)==str:
        if "GB" in x :
            x = x.replace("GB","")
            x= float(x)*1024
        elif "G" in x:
            x = x.replace("G","")
            x= float(x)*1024
        elif "MB" in x:
            x = x.replace("MB","")
            x= float(x)
        elif "TB" in x:
            x = x.replace("TB","")
            x = float(x)*1024*1024
    elif math.isnan(float(x)):
        pass
    return x

def en_giga(x):
    return x/1024

In [19]:
basepath_ibm_volume ='C:/Users/lsarlat/Documents/Capacity_planning/CP3/données_version3/ibm/volume/'
df_ibm_volume = pd.DataFrame()
for name in fichiers_ibm_a_recuperer:
    with open(os.path.join(basepath_ibm_volume, name)) as f:
        df_temp = pd.read_fwf(f)
        df_temp['date']= datetime.datetime.strptime(str(int(name[:8])),'%Y%m%d')
        df_ibm_volume = pd.concat([df_ibm_volume, df_temp])
df_ibm_volume

,vdisk_id,vdisk_name,copy_id,mdisk_grp_id,mdisk_grp_name,capacity,used_capacity,real_capacity,free_capacity,overallocation,...,grainsize,se_copy,compressed_copy,uncompressed_used_capacity,parent_mdisk_grp_id,parent_mdisk_grp_name,encrypt,deduplicated_copy,used_capacity_before_reduction,date
0,0,MUTN_19DC1_T0S_ORASIP_DATA_01,0,8,SDC1_T0,400.00GB,170.08GB,171.89GB,1.81GB,232,...,256.0,yes,no,170.08GB,8,SDC1_T0,yes,no,NaN,2021-03-18
1,1,CMCN_19DC1_T2S_ORACMCF_DATA_FG1_04,0,6,SDC1_T2_FCM,200.00GB,199.80GB,200.01GB,217.25MB,99,...,256.0,yes,no,199.80GB,6,SDC1_T2_FCM,yes,no,NaN,2021-03-18
2,2,CMCN_19DC1_T2S_ORACMCF_DATA_FG1_05,1,6,SDC1_T2_FCM,200.00GB,199.80GB,200.01GB,211.75MB,99,...,256.0,yes,no,199.80GB,6,SDC1_T2_FCM,yes,no,NaN,2021-03-18
3,3,MUTN_19DC3_T0S_ORASIP_DATA_01,0,1,SDC3_T0,400.00GB,170.08GB,171.89GB,1.81GB,232,...,256.0,yes,no,170.08GB,1,SDC3_T0,yes,no,NaN,2021-03-18
4,4,CMCN_19DC1_T2S_ORACMCF_DATA_FG1_06,1,6,SDC1_T2_FCM,200.00GB,199.80GB,200.01GB,218.25MB,99,...,256.0,yes,no,199.80GB,6,SDC1_T2_FCM,yes,no,NaN,2021-03-18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
933,768,CTZS_19DC1_T2M_CTZS_01_DS_02,1,7,SDC3_T2_FCM,3.00TB,149.96GB,211.41GB,61.45GB,1453,...,256.0,yes,no,149.96GB,7,SDC3_T2_FCM,yes,no,NaN,2021-03-18
934,769,CTZS_19DC3_T2M_CTZS_01_DS_02,0,7,SDC3_T2_FCM,3.00TB,177.25MB,82.12GB,81.95GB,3740,...,256.0,yes,no,177.25MB,7,SDC3_T2_FCM,yes,no,NaN,2021-03-18
935,769,CTZS_19DC3_T2M_CTZS_01_DS_02,1,6,SDC1_T2_FCM,3.00TB,177.25MB,82.11GB,81.93GB,3741,...,256.0,yes,no,177.25MB,6,SDC1_T2_FCM,yes,no,NaN,2021-03-18
936,770,CR2N_19DC1_T2S_DS_01,0,6,SDC1_T2_FCM,256.00GB,166.41GB,171.54GB,5.13GB,149,...,256.0,yes,no,166.41GB,6,SDC1_T2_FCM,yes,no,NaN,2021-03-18


In [20]:
#suppression des lignes où la used_capcity est null
df_ibm_volume = df_ibm_volume[df_ibm_volume.used_capacity.notnull()]
for colonne in ["capacity", "used_capacity","real_capacity", "free_capacity", "uncompressed_used_capacity", "used_capacity_before_reduction"]:
    df_ibm_volume[colonne] = df_ibm_volume[colonne].map(lambda x : en_MB(x) )
    
df_ibm_volume = df_ibm_volume.rename(columns={"vdisk_id":"ID","vdisk_name":"name","mdisk_grp_name": "pool"})
df_ibm_volume['offre'] = df_ibm_volume.pool
df_ibm_volume.offre= df_ibm_volume.offre.map(lambda x: x[5:])
df_ibm_volume["origine"] = np.array("ibm")
df_ibm_volume

,ID,name,copy_id,mdisk_grp_id,pool,capacity,used_capacity,real_capacity,free_capacity,overallocation,...,compressed_copy,uncompressed_used_capacity,parent_mdisk_grp_id,parent_mdisk_grp_name,encrypt,deduplicated_copy,used_capacity_before_reduction,date,offre,origine
0,0,MUTN_19DC1_T0S_ORASIP_DATA_01,0,8,SDC1_T0,409600.0,174161.92,176015.36,1853.44,232,...,no,174161.92,8,SDC1_T0,yes,no,NaN,2021-03-18,T0,ibm
1,1,CMCN_19DC1_T2S_ORACMCF_DATA_FG1_04,0,6,SDC1_T2_FCM,204800.0,204595.20,204810.24,217.25,99,...,no,204595.20,6,SDC1_T2_FCM,yes,no,NaN,2021-03-18,T2_FCM,ibm
2,2,CMCN_19DC1_T2S_ORACMCF_DATA_FG1_05,1,6,SDC1_T2_FCM,204800.0,204595.20,204810.24,211.75,99,...,no,204595.20,6,SDC1_T2_FCM,yes,no,NaN,2021-03-18,T2_FCM,ibm
3,3,MUTN_19DC3_T0S_ORASIP_DATA_01,0,1,SDC3_T0,409600.0,174161.92,176015.36,1853.44,232,...,no,174161.92,1,SDC3_T0,yes,no,NaN,2021-03-18,T0,ibm
4,4,CMCN_19DC1_T2S_ORACMCF_DATA_FG1_06,1,6,SDC1_T2_FCM,204800.0,204595.20,204810.24,218.25,99,...,no,204595.20,6,SDC1_T2_FCM,yes,no,NaN,2021-03-18,T2_FCM,ibm
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
933,768,CTZS_19DC1_T2M_CTZS_01_DS_02,1,7,SDC3_T2_FCM,3145728.0,153559.04,216483.84,62924.80,1453,...,no,153559.04,7,SDC3_T2_FCM,yes,no,NaN,2021-03-18,T2_FCM,ibm
934,769,CTZS_19DC3_T2M_CTZS_01_DS_02,0,7,SDC3_T2_FCM,3145728.0,177.25,84090.88,83916.80,3740,...,no,177.25,7,SDC3_T2_FCM,yes,no,NaN,2021-03-18,T2_FCM,ibm
935,769,CTZS_19DC3_T2M_CTZS_01_DS_02,1,6,SDC1_T2_FCM,3145728.0,177.25,84080.64,83896.32,3741,...,no,177.25,6,SDC1_T2_FCM,yes,no,NaN,2021-03-18,T2_FCM,ibm
936,770,CR2N_19DC1_T2S_DS_01,0,6,SDC1_T2_FCM,262144.0,170403.84,175656.96,5253.12,149,...,no,170403.84,6,SDC1_T2_FCM,yes,no,NaN,2021-03-18,T2_FCM,ibm


In [21]:
df_ibm_volume_reduit = df_ibm_volume[["ID","name","pool","offre","capacity","used_capacity",'date','origine']].copy()

df_ibm_volume_reduit.reset_index(drop=True,inplace =True)

df_ibm_volume_GB = df_ibm_volume_reduit.copy()
for colonne in ["capacity", "used_capacity"]:
     df_ibm_volume_GB[colonne] = df_ibm_volume_reduit[colonne].map(lambda x : en_giga(x)).copy()
df_ibm_volume_GB

,ID,name,pool,offre,capacity,used_capacity,date,origine
0,0,MUTN_19DC1_T0S_ORASIP_DATA_01,SDC1_T0,T0,400.0,170.080000,2021-03-18,ibm
1,1,CMCN_19DC1_T2S_ORACMCF_DATA_FG1_04,SDC1_T2_FCM,T2_FCM,200.0,199.800000,2021-03-18,ibm
2,2,CMCN_19DC1_T2S_ORACMCF_DATA_FG1_05,SDC1_T2_FCM,T2_FCM,200.0,199.800000,2021-03-18,ibm
3,3,MUTN_19DC3_T0S_ORASIP_DATA_01,SDC3_T0,T0,400.0,170.080000,2021-03-18,ibm
4,4,CMCN_19DC1_T2S_ORACMCF_DATA_FG1_06,SDC1_T2_FCM,T2_FCM,200.0,199.800000,2021-03-18,ibm
...,...,...,...,...,...,...,...,...
933,768,CTZS_19DC1_T2M_CTZS_01_DS_02,SDC3_T2_FCM,T2_FCM,3072.0,149.960000,2021-03-18,ibm
934,769,CTZS_19DC3_T2M_CTZS_01_DS_02,SDC3_T2_FCM,T2_FCM,3072.0,0.173096,2021-03-18,ibm
935,769,CTZS_19DC3_T2M_CTZS_01_DS_02,SDC1_T2_FCM,T2_FCM,3072.0,0.173096,2021-03-18,ibm
936,770,CR2N_19DC1_T2S_DS_01,SDC1_T2_FCM,T2_FCM,256.0,166.410000,2021-03-18,ibm


In [22]:
def typo(serveur):
    if "APP" in serveur:
        typo = "APP"
    elif "WEB" in serveur:
        typo = "WEB"
    elif "DAT" in serveur:
        typo = "DAT"
    elif "SQL" in serveur:
        typo = "DAT"
    elif "ORA" in serveur:
        typo = "DAT"
    elif "VIR" in serveur:
        typo = "VIR"
    elif "vir" in serveur:
        typo = "VIR"
    elif "FIC" in serveur:
        typo = "FIC"
    elif "MES" in serveur:
        typo = "MES"
    elif "DNS" in serveur:
        typo = "DNS"
    else:
        typo = "AUTRE"
 
    return typo

In [23]:
# trouver le fichier server le plus recent
#basepath_ibm_server ='D:/Capacity_Planing/Stockage/ibm/server/'
basepath_ibm_server ='/Users/lsarlat/Documents/Capacity_planning/CP3/données_version3/ibm/server/'
fichiers_ibm_server=[]
with os.scandir(basepath_ibm_server) as entries: # renvoie un itérateur
    for entry in entries:
        if entry.is_file():
            fichiers_ibm_server.append(str(entry.name))
            
date_max = datetime.datetime(1999,1,1)
fichier_recent =''
for i in fichiers_ibm_server:
    d = datetime.datetime.strptime(str(int(i[:8])),'%Y%m%d')
    if d > date_max:
        date_max = d
        fichier_recent = i
print(date_max, fichier_recent)

2021-03-18 00:00:00 20210318_mutsapsan019_lshostvdiskmap.txt


In [24]:
basepath='C:/Users/lsarlat/Documents/Capacity_planning/CP3/données_version3/ibm/server/'
df_ibm_server = pd.DataFrame()
for name in [fichier_recent]:
    with open(os.path.join(basepath, name)) as f:
        df_temp = pd.read_csv(f, delim_whitespace =True, names =['ID', 'name_server',"SCSI_id" ,"vdisk_id" ,"vdisk_name" ,"vdisk_UID","IO_group_id","IO_group_name", "mapping_type","host_cluster_id","host_cluster_name"])
        df_temp['date']= datetime.datetime.strptime(str(int(name[:8])),'%Y%m%d')
        df_ibm_server = pd.concat([df_ibm_server, df_temp])

df_ibm_server.reset_index(drop=True,inplace =True)
# supprime les lignes où le name n'est pas bon.
df_ibm_server = df_ibm_server.drop(df_ibm_server.name_server[df_ibm_server.name_server == "name"].index)
df_ibm_server = df_ibm_server.drop(df_ibm_server.name_server[df_ibm_server.vdisk_name.isnull()].index)

# création d'une colonne typo
df_ibm_server['typologie'] = df_ibm_server.name_server
df_ibm_server.typologie = df_ibm_server.typologie.map(lambda x : typo(x))
df_ibm_server

,ID,name_server,SCSI_id,vdisk_id,vdisk_name,vdisk_UID,IO_group_id,IO_group_name,mapping_type,host_cluster_id,host_cluster_name,date,typologie
2,0,EXLIDAT1,0,428,INFN_19DC3_T2M_EXLIDAT1_TEST_01,600507680C80856BD8000000000001D0,0,io_grp0,private,NaN,NaN,2021-03-18,DAT
3,0,EXLIDAT1,3,12,MUTN_19DC1_T2S_BACULA_CATALOGUE_01,600507680C80856BD80000000000000E,0,io_grp0,private,NaN,NaN,2021-03-18,DAT
4,0,EXLIDAT1,1,11,MUTN_19DC1_T2S_BACULA_PGSQL_01,600507680C80856BD80000000000000D,1,io_grp1,private,NaN,NaN,2021-03-18,DAT
7,1,EXLIDAT2,1,10,MUTN_19DC3_T2S_BACULA_PGSQL_01,600507680C80856BD800000000000014,0,io_grp0,private,NaN,NaN,2021-03-18,DAT
8,1,EXLIDAT2,3,9,MUTN_19DC3_T2S_BACULA_CATALOGUE_01,600507680C80856BD800000000000013,1,io_grp1,private,NaN,NaN,2021-03-18,DAT
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1595700,204,PICSVPVIR002,3,750,PICS_19DC3_T2M_PICS_01_DS_01,600507680C80856BD800000000000880,2,io_grp2,shared,55,CL_CLOUD_P_PICS_01,2021-03-18,VIR
1595703,206,PRVNVPVIR002,1,754,PRVN_19DC3_T2M_PRVN_01_DS_01,600507680C80856BD800000000000886,1,io_grp1,shared,56,CL_CLOUD_P_PRVN_01,2021-03-18,VIR
1595704,206,PRVNVPVIR002,0,753,PRVN_19DC1_T2M_PRVN_01_DS_01,600507680C80856BD800000000000885,2,io_grp2,shared,56,CL_CLOUD_P_PRVN_01,2021-03-18,VIR
1595707,208,PRVNVPVIR001,1,754,PRVN_19DC3_T2M_PRVN_01_DS_01,600507680C80856BD800000000000886,1,io_grp1,shared,56,CL_CLOUD_P_PRVN_01,2021-03-18,VIR


In [25]:
#Création d'un df pour connaitre la typologie du volume et le nom de client(si possible) à partir d'un nom de server associé
liste_volume = []
liste_type = []
liste_client =[]
for volume in df_ibm_server.vdisk_name.unique():
    if volume in liste_volume:
        pass
    else:
        liste_volume.append(volume)
        liste_client.append(df_ibm_server[df_ibm_server.vdisk_name == volume].name_server.unique()[0][:3])
        liste_type.append(df_ibm_server[df_ibm_server.vdisk_name == volume].typologie.unique()[0])
df_t = pd.DataFrame({"name_v":liste_volume,"client":liste_client,"type":liste_type})
df_t

,name_v,client,type
0,INFN_19DC3_T2M_EXLIDAT1_TEST_01,EXL,DAT
1,MUTN_19DC1_T2S_BACULA_CATALOGUE_01,EXL,DAT
2,MUTN_19DC1_T2S_BACULA_PGSQL_01,EXL,DAT
3,MUTN_19DC3_T2S_BACULA_PGSQL_01,EXL,DAT
4,MUTN_19DC3_T2S_BACULA_CATALOGUE_01,EXL,DAT
...,...,...,...
1031,TEMP_INFSVPVIR030_SDC1_T2,INF,VIR
1032,CTZS_19DC1_T2M_CTZS_01_DS_02,CTZ,VIR
1033,CTZS_19DC3_T2M_CTZS_01_DS_02,CTZ,VIR
1034,CR2N_19DC1_T2S_DS_01,MUT,VIR


In [26]:
# Etiquetage en typologie et nom des clients des volumes ibm à partir du dataframe constitué précédemment df_t.
# Si le serveur n'est reconnu, on tente de trouver la typo via son nom avec la fonction typo()
df_ibm_volume_plus = df_ibm_volume_GB.copy()

liste_typo =[]
liste_client=[]

for v in df_ibm_volume_plus.name:
    if v in df_t.name_v.unique().tolist(): # récupération de la typologie via le df df_t
        liste_typo.append(df_t[df_t.name_v == v].type.values[0])
        liste_client.append(df_t[df_t.name_v == v].client.values[0])
    else:
        if v[:3]=="INF":
            liste_typo.append(typo(v))   # appel de la fonction typo() pour trouver la typologie
            liste_client.append('INF')
        elif v[:3]=="SIG":
            liste_typo.append(typo(v))   
            liste_client.append('SIG')
        else :
            liste_typo.append(typo(v)) 
            liste_client.append('Client_non_enregistré')
df_ibm_volume_plus['typologie'] = liste_typo
df_ibm_volume_plus['client'] = liste_client
df_ibm_volume_plus

,ID,name,pool,offre,capacity,used_capacity,date,origine,typologie,client
0,0,MUTN_19DC1_T0S_ORASIP_DATA_01,SDC1_T0,T0,400.0,170.080000,2021-03-18,ibm,DAT,MUT
1,1,CMCN_19DC1_T2S_ORACMCF_DATA_FG1_04,SDC1_T2_FCM,T2_FCM,200.0,199.800000,2021-03-18,ibm,DAT,CMC
2,2,CMCN_19DC1_T2S_ORACMCF_DATA_FG1_05,SDC1_T2_FCM,T2_FCM,200.0,199.800000,2021-03-18,ibm,DAT,CMC
3,3,MUTN_19DC3_T0S_ORASIP_DATA_01,SDC3_T0,T0,400.0,170.080000,2021-03-18,ibm,DAT,MUT
4,4,CMCN_19DC1_T2S_ORACMCF_DATA_FG1_06,SDC1_T2_FCM,T2_FCM,200.0,199.800000,2021-03-18,ibm,DAT,CMC
...,...,...,...,...,...,...,...,...,...,...
933,768,CTZS_19DC1_T2M_CTZS_01_DS_02,SDC3_T2_FCM,T2_FCM,3072.0,149.960000,2021-03-18,ibm,VIR,CTZ
934,769,CTZS_19DC3_T2M_CTZS_01_DS_02,SDC3_T2_FCM,T2_FCM,3072.0,0.173096,2021-03-18,ibm,VIR,CTZ
935,769,CTZS_19DC3_T2M_CTZS_01_DS_02,SDC1_T2_FCM,T2_FCM,3072.0,0.173096,2021-03-18,ibm,VIR,CTZ
936,770,CR2N_19DC1_T2S_DS_01,SDC1_T2_FCM,T2_FCM,256.0,166.410000,2021-03-18,ibm,VIR,MUT


In [27]:
# 3par

basepath_3par_volume = 'C:/Users/lsarlat/Documents/Capacity_planning/CP3/données_version3/3par/volume'
# List all files in a directory using scandir()
liste_3par_volume=[]
with os.scandir(basepath_3par_volume) as entries: # renvoie un itérateur
    for entry in entries:
        if entry.is_file():
            liste_3par_volume.append(str(entry.name))
print(len(liste_3par_volume))

2250


In [28]:
fichiers_3par_a_recuperer = []
for i in liste_3par_volume:
    if i not in df_fichiers.nom_fichier.to_list():
        fichiers_3par_a_recuperer.append(i)
fichiers_3par_a_recuperer

[]

In [29]:
# pour le test
fichiers_3par_a_recuperer =["20210317_mutapsan06_showvv-space.txt"]

In [30]:
# création du DF principal à partir de la liste_3par_volume
df_3par_volume = pd.DataFrame()
for name in fichiers_3par_a_recuperer:
    with open(os.path.join(basepath_3par_volume, name)) as f:
        # gestion des formats de fichiers avant / après le 08 janvier 2020
        if datetime.datetime.strptime(str(int(name[:8])),'%Y%m%d')<= datetime.datetime(2020,1,8):
            df_temp = pd.read_csv(f,delim_whitespace =True,  names= ["ID","name","Prov","Type","adm_Rsvd","adm_Used","snp_Rsvd","snp_Used","%_snp_Used","snp_Wrn","snp_Lim","usr_Rsvd" ,"usr_Used","%usr_Used", "usr_Wrn" ,"usr_Lim","Tot_Rsvd","VSize","Compaction","Dedup"], header = 4)
            df_temp['date']= datetime.datetime.strptime(str(int(name[:8])),'%Y%m%d')
            df_temp['pool']= name[9:19]
            df_temp = df_temp.rename(columns={"Tot_Rsvd": "Tot_used"})
        else:
            df_temp = pd.read_csv(f,delim_whitespace =True,  names= ["ID","name","Prov","Type","adm_Rsvd","adm_Used","snp_Rsvd","snp_Used","%_snp_Used","snp_Wrn","snp_Lim","usr_Rsvd" ,"usr_Used","%usr_Used", "usr_Wrn" ,"usr_Lim","Tot_Rsvd","Tot_used","HostWR","VSize","Compact","Compress"], header = 4)
            df_temp['date']= datetime.datetime.strptime(str(int(name[:8])),'%Y%m%d')
            df_temp['pool']= name[9:19]
        df_3par_volume = pd.concat([df_3par_volume, df_temp])
        
# supprime les lignes où le name n'est pas bon.        
df_3par_volume = df_3par_volume.drop(df_3par_volume.name[df_3par_volume.name.isnull()].index)
df_3par_volume = df_3par_volume.drop(df_3par_volume.name[df_3par_volume.name=='total'].index)
df_3par_volume = df_3par_volume[df_3par_volume.name !="No"]
# renommage des colonnes
df_3par_volume = df_3par_volume.rename(columns={"Tot_used": "used_capacity", "VSize":"capacity"})

# limitation du nb de colonnes
df_3par_volume = df_3par_volume[["ID","name","capacity","used_capacity","date","pool"]]
df_3par_volume['offre'] = np.array('T2')
#suppression des lignes où la used_capcity est null
df_3par_volume = df_3par_volume[df_3par_volume.used_capacity.notnull()]

# réinitialise les index
df_3par_volume.reset_index(drop=True,inplace =True)

# traitement sur la colonne date
df_3par_volume.date = df_3par_volume.date.map(lambda x :  datetime.datetime.strptime(str(x)[:10],'%Y-%m-%d'))

# ajout d'une colonne origine
df_3par_volume['origine'] = np.array("3par")
# ajout de colonne Client
df_3par_volume['client'] = df_3par_volume.name
df_3par_volume.client = df_3par_volume.client.map(lambda x : x[:3])
df_3par_volume

,ID,name,capacity,used_capacity,date,pool,offre,origine,client
0,111,GRCS_06DC3_T2S_GRCS_01_DS_01,7340032.0,6269211.0,2021-03-17,mutapsan06,T2,3par,GRC
1,61,IPSS_06T2S_SQL_02,2048000.0,1538537.0,2021-03-17,mutapsan06,T2,3par,IPS
2,108,LEFN_06T2N_IBM_DATA_02,112640.0,3926.0,2021-03-17,mutapsan06,T2,3par,LEF
3,109,LEFN_06T2N_IBM_OS_02,20480.0,9423.0,2021-03-17,mutapsan06,T2,3par,LEF
4,46,LEFN_06T2S_IBM_DATA_01,1335296.0,1080977.0,2021-03-17,mutapsan06,T2,3par,LEF
5,45,LEFN_06T2S_IBM_OS_01,40960.0,11697.0,2021-03-17,mutapsan06,T2,3par,LEF
6,106,MLHS_06DC3_T2S_MLHS_01_DS_01,716800.0,697605.0,2021-03-17,mutapsan06,T2,3par,MLH
7,97,MUTN_06DC3_T2S_MUTN_01_DS_01,4194304.0,3709870.0,2021-03-17,mutapsan06,T2,3par,MUT
8,103,MUTN_06DC3_T2S_MUTN_01_DS_02,2097152.0,1607126.0,2021-03-17,mutapsan06,T2,3par,MUT
9,98,MUTN_06DC3_T2S_MUTN_02_DS_01,4194304.0,3733933.0,2021-03-17,mutapsan06,T2,3par,MUT


In [31]:
# conversion en GB
df_3par_volume_GB = df_3par_volume.copy()
for colonne in ['capacity','used_capacity']:
    df_3par_volume_GB[colonne] = df_3par_volume_GB[colonne].map(lambda x :en_giga(x))
df_3par_volume_GB

,ID,name,capacity,used_capacity,date,pool,offre,origine,client
0,111,GRCS_06DC3_T2S_GRCS_01_DS_01,7168.0,6122.276367,2021-03-17,mutapsan06,T2,3par,GRC
1,61,IPSS_06T2S_SQL_02,2000.0,1502.477539,2021-03-17,mutapsan06,T2,3par,IPS
2,108,LEFN_06T2N_IBM_DATA_02,110.0,3.833984,2021-03-17,mutapsan06,T2,3par,LEF
3,109,LEFN_06T2N_IBM_OS_02,20.0,9.202148,2021-03-17,mutapsan06,T2,3par,LEF
4,46,LEFN_06T2S_IBM_DATA_01,1304.0,1055.641602,2021-03-17,mutapsan06,T2,3par,LEF
5,45,LEFN_06T2S_IBM_OS_01,40.0,11.422852,2021-03-17,mutapsan06,T2,3par,LEF
6,106,MLHS_06DC3_T2S_MLHS_01_DS_01,700.0,681.254883,2021-03-17,mutapsan06,T2,3par,MLH
7,97,MUTN_06DC3_T2S_MUTN_01_DS_01,4096.0,3622.919922,2021-03-17,mutapsan06,T2,3par,MUT
8,103,MUTN_06DC3_T2S_MUTN_01_DS_02,2048.0,1569.458984,2021-03-17,mutapsan06,T2,3par,MUT
9,98,MUTN_06DC3_T2S_MUTN_02_DS_01,4096.0,3646.418945,2021-03-17,mutapsan06,T2,3par,MUT


In [32]:
# trouver le fichier server le plus recent
#basepath_3par_server ='D:/Capacity_Planing/Stockage/3par/server/showlun'
basepath_3par_server ='/Users/lsarlat/Documents/Capacity_planning/CP3/données_version3/3par/server/lun'
fichiers_3par_server=[]
with os.scandir(basepath_3par_server) as entries: # renvoie un itérateur
    for entry in entries:
        if entry.is_file():
            fichiers_3par_server.append(str(entry.name))
            
date_max = datetime.datetime(1999,1,1)
fichier_server_3par_recent =''
for i in fichiers_3par_server:
    d = datetime.datetime.strptime(str(int(i[:8])),'%Y%m%d')
    if d > date_max:
        date_max = d
        ladate = str(date_max)[:10].replace('-','')
        fichier_server_3par_recent = [ladate + "_mutapsan05_showvlun-lvw.txt", ladate +"_mutapsan05_showvlun-lvw.txt"]

print(date_max, fichier_server_3par_recent)

2021-03-17 00:00:00 ['20210317_mutapsan05_showvlun-lvw.txt', '20210317_mutapsan05_showvlun-lvw.txt']


In [33]:
# création du df Server volume avec un fichier server 3par (le plus récent ) 
basepath_3par_server = 'C:/Users/lsarlat/Documents/Capacity_planning/CP3/données_version3/3par/server/lun/'
df_3par_server = pd.DataFrame()
for name in fichier_server_3par_recent:
    with open(os.path.join(basepath_3par_server, name)) as f:
        df_temp = pd.read_csv(f, delim_whitespace =True, names =['ID_volume', 'name_volume',"vv_wwn" ,"server" ,"Host_WWN/ISCSI" ,"port","Type","Status", "id"])
        df_temp['date']= name[:8]
        df_3par_server = pd.concat([df_3par_server, df_temp])
    
# réinitialise les index
df_3par_server.reset_index(drop=True,inplace =True)
        
# supprime les lignes où le name n'est pas bon.
df_3par_server = df_3par_server.drop(df_3par_server.name_volume[df_3par_server.name_volume == "VVName"].index)
df_3par_server = df_3par_server.drop(df_3par_server.name_volume[df_3par_server.name_volume == "VLUNs"].index)
df_3par_server = df_3par_server.drop(df_3par_server.name_volume[df_3par_server.ID_volume == "VLUN"].index)
df_3par_server = df_3par_server.drop(df_3par_server.name_volume[df_3par_server.name_volume == "total"].index)
df_3par_server = df_3par_server.drop(df_3par_server.name_volume[df_3par_server['Host_WWN/ISCSI'] == "----------------"].index)
df_3par_server = df_3par_server.drop(df_3par_server.name_volume[df_3par_server.name_volume.isnull()].index)
df_3par_server

,ID_volume,name_volume,vv_wwn,server,Host_WWN/ISCSI,port,Type,Status,id,date
2,0,IAXI_05T2S_OVM_01,60002AC0000000000000002B00019825,INFNVQVIR001,10008666E8200084,0:0:1,host,set,active,20210317
3,1,IAXI_05T2S_OVM-RAC01_01,60002AC0000000000000002C00019825,INFNVQVIR001,10008666E8200084,0:0:1,host,set,active,20210317
4,2,IAXI_05T2S_OVM-RAC01_02,60002AC0000000000000002D00019825,INFNVQVIR001,10008666E8200084,0:0:1,host,set,active,20210317
5,3,IAXI_05T2S_OVM-RAC01_03,60002AC0000000000000002E00019825,INFNVQVIR001,10008666E8200084,0:0:1,host,set,active,20210317
6,4,IAXI_05T2S_OVM-RAC01_04,60002AC0000000000000002F00019825,INFNVQVIR001,10008666E8200084,0:0:1,host,set,active,20210317
...,...,...,...,...,...,...,...,...,...,...
1805,1,SIGN_05DC1_T2S_SIGS_01_DS_02,60002AC0000000000000007300019825,SIGVPVIR13,21000024FF914893,1:0:2,host,set,active,20210317
1806,1,SIGN_05DC1_T2S_SIGS_01_DS_02,60002AC0000000000000007300019825,SIGVPVIR14,21000024FF9148A6,0:0:1,host,set,active,20210317
1807,1,SIGN_05DC1_T2S_SIGS_01_DS_02,60002AC0000000000000007300019825,SIGVPVIR14,21000024FF9148A7,0:0:2,host,set,active,20210317
1808,1,SIGN_05DC1_T2S_SIGS_01_DS_02,60002AC0000000000000007300019825,SIGVPVIR14,21000024FF9148A6,1:0:1,host,set,active,20210317


In [34]:
df_3par_server['typologie'] = df_3par_server.server
df_3par_server.typologie = df_3par_server.typologie.map(lambda x : typo(x))

In [35]:
# étiquetage de la typologie des volumes 3par
df_3par_volume_plus = df_3par_volume_GB.copy()
liste_typo =[]
for v in df_3par_volume_plus.name:
    if v in df_3par_server.name_volume.unique().tolist():
        liste_typo.append(df_3par_server[df_3par_server.name_volume == v].typologie.values[0])
    else:
        liste_typo.append(typo(v))
        
df_3par_volume_plus['typologie'] = liste_typo
df_3par_volume_plus

,ID,name,capacity,used_capacity,date,pool,offre,origine,client,typologie
0,111,GRCS_06DC3_T2S_GRCS_01_DS_01,7168.0,6122.276367,2021-03-17,mutapsan06,T2,3par,GRC,AUTRE
1,61,IPSS_06T2S_SQL_02,2000.0,1502.477539,2021-03-17,mutapsan06,T2,3par,IPS,DAT
2,108,LEFN_06T2N_IBM_DATA_02,110.0,3.833984,2021-03-17,mutapsan06,T2,3par,LEF,DAT
3,109,LEFN_06T2N_IBM_OS_02,20.0,9.202148,2021-03-17,mutapsan06,T2,3par,LEF,AUTRE
4,46,LEFN_06T2S_IBM_DATA_01,1304.0,1055.641602,2021-03-17,mutapsan06,T2,3par,LEF,DAT
5,45,LEFN_06T2S_IBM_OS_01,40.0,11.422852,2021-03-17,mutapsan06,T2,3par,LEF,AUTRE
6,106,MLHS_06DC3_T2S_MLHS_01_DS_01,700.0,681.254883,2021-03-17,mutapsan06,T2,3par,MLH,AUTRE
7,97,MUTN_06DC3_T2S_MUTN_01_DS_01,4096.0,3622.919922,2021-03-17,mutapsan06,T2,3par,MUT,AUTRE
8,103,MUTN_06DC3_T2S_MUTN_01_DS_02,2048.0,1569.458984,2021-03-17,mutapsan06,T2,3par,MUT,AUTRE
9,98,MUTN_06DC3_T2S_MUTN_02_DS_01,4096.0,3646.418945,2021-03-17,mutapsan06,T2,3par,MUT,AUTRE


In [106]:
# union des df ibm et 3PAR
df_volume = pd.concat([df_ibm_volume_plus,df_3par_volume_plus])
df_volume.date = df_volume.date.map(lambda x :  datetime.datetime.strptime(str(x)[:10],'%Y-%m-%d'))
df_volume = df_volume.rename(columns ={'ID':'vdisk_id','name':'name_volume','offre':'name_offre', 'pool':'name_pool','typologie':'name_typologie'})

new_index = np.arange(start=(last_index_id_enreg+1), stop=(last_index_id_enreg+1)+len(df_volume), step=1)
df_volume.index = new_index
df_volume = df_volume.reset_index()
df_volume = df_volume.rename(columns={"index": "id_enreg"})
df_volume

,id_enreg,vdisk_id,name_volume,name_pool,name_offre,capacity,used_capacity,date,origine,name_typologie,client
0,681443,0,MUTN_19DC1_T0S_ORASIP_DATA_01,SDC1_T0,T0,400.0,170.080000,2021-03-18,ibm,DAT,MUT
1,681444,1,CMCN_19DC1_T2S_ORACMCF_DATA_FG1_04,SDC1_T2_FCM,T2_FCM,200.0,199.800000,2021-03-18,ibm,DAT,CMC
2,681445,2,CMCN_19DC1_T2S_ORACMCF_DATA_FG1_05,SDC1_T2_FCM,T2_FCM,200.0,199.800000,2021-03-18,ibm,DAT,CMC
3,681446,3,MUTN_19DC3_T0S_ORASIP_DATA_01,SDC3_T0,T0,400.0,170.080000,2021-03-18,ibm,DAT,MUT
4,681447,4,CMCN_19DC1_T2S_ORACMCF_DATA_FG1_06,SDC1_T2_FCM,T2_FCM,200.0,199.800000,2021-03-18,ibm,DAT,CMC
...,...,...,...,...,...,...,...,...,...,...,...
959,682402,71,SNIN_06T2S_ORASNI_DATA_FG2_03,mutapsan06,T2,200.0,170.034180,2021-03-17,3par,DAT,SNI
960,682403,72,SNIN_06T2S_ORASNI_DATA_FG2_04,mutapsan06,T2,200.0,170.041016,2021-03-17,3par,DAT,SNI
961,682404,68,SNIN_06T2S_ORASNI_FRA_FG2_01,mutapsan06,T2,20.0,10.547852,2021-03-17,3par,DAT,SNI
962,682405,66,SNIN_06T2S_ORASNI_OCR_01_SDC3,mutapsan06,T2,0.5,0.008789,2021-03-17,3par,DAT,SNI


In [107]:
query_read_offres = "SELECT * FROM stk_offres"
cur.execute(query_read_offres)
raw_offres = cur.fetchall()
offres = pd.DataFrame(raw_offres, columns =["id_offre","name_offre"])
offres

,id_offre,name_offre
0,0,T0
1,1,T2
2,2,T2_V7000
3,3,T0_V7000
4,4,T2_TMP
5,5,T2_FCM


In [108]:
query_read_pools = "SELECT * FROM stk_pools"
cur.execute(query_read_pools)
raw_pools = cur.fetchall()
pools = pd.DataFrame(raw_pools, columns =["id_pool","name_pool","id_offre"])
pools

,id_pool,name_pool,id_offre
0,0,SDC3_T0,0
1,1,SDC1_T0,0
2,2,SDC1_T2,1
3,3,SDC3_T2,1
4,11,mutapsan05,1
5,12,mutapsan06,1
6,4,SDC1_T2_V7000,2
7,6,SDC3_T2_V7000,2
8,5,SDC3_T0_V7000,3
9,7,SDC3_T2_TMP,4


In [109]:
query_read_typologie = "SELECT * FROM stk_typologies"
cur.execute(query_read_typologie)
raw_typologies = cur.fetchall()
typologies = pd.DataFrame(raw_typologies, columns =["id_typologie","name_typologie"])
typologies

,id_typologie,name_typologie
0,0,DAT
1,1,AUTRE
2,2,VIR
3,3,WEB
4,4,APP


In [110]:
query_read_origines = "SELECT * FROM stk_origines"
cur.execute(query_read_origines)
raw_origines = cur.fetchall()

origines = pd.DataFrame(raw_origines, columns =["id_origine","origine"])
origines

,id_origine,origine
0,0,ibm
1,1,3par


In [111]:
df_volume = df_volume.merge(pools, on='name_pool')
df_volume = df_volume.merge(typologies, on='name_typologie')
df_volume = df_volume.merge(origines, on='origine')
df_volume

,id_enreg,vdisk_id,name_volume,name_pool,name_offre,capacity,used_capacity,date,origine,name_typologie,client,id_pool,id_offre,id_typologie,id_origine
0,681443,0,MUTN_19DC1_T0S_ORASIP_DATA_01,SDC1_T0,T0,400.0,170.080000,2021-03-18,ibm,DAT,MUT,1,0,0,0
1,681464,20,ELEN_19DC1_T0S_ORAD01_DATA_01,SDC1_T0,T0,25.0,8.730000,2021-03-18,ibm,DAT,ELE,1,0,0,0
2,681465,21,ELEN_19DC1_T0S_ORAD01_DATA_02,SDC1_T0,T0,25.0,8.720000,2021-03-18,ibm,DAT,ELE,1,0,0,0
3,681466,22,ELEN_19DC1_T0S_ORAD01_DATA_03,SDC1_T0,T0,25.0,8.720000,2021-03-18,ibm,DAT,ELE,1,0,0,0
4,681467,23,ELEN_19DC1_T0S_ORAD01_DATA_04,SDC1_T0,T0,25.0,8.730000,2021-03-18,ibm,DAT,ELE,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
959,682394,21,MUTS_06DC3_T2S_MUTS_01_DS_02,mutapsan06,T2,4096.0,3951.434570,2021-03-17,3par,AUTRE,MUT,12,1,1,1
960,682395,110,MUTS_06DC3_T2S_MUTS_01_DS_03,mutapsan06,T2,4096.0,3175.658203,2021-03-17,3par,AUTRE,MUT,12,1,1,1
961,682396,22,MUTS_06DC3_T2S_MUTS_01_DS_05,mutapsan06,T2,4096.0,3833.631836,2021-03-17,3par,AUTRE,MUT,12,1,1,1
962,682397,78,SIGN_06DC3_T2S_SIGS_01_DS_01,mutapsan06,T2,5120.0,4141.827148,2021-03-17,3par,AUTRE,SIG,12,1,1,1


In [91]:
import psycopg2
import psycopg2.extras as extras

param_dic = {
    "host"      : "localhost",
    "database"  : "cp2",
    "user"      : "postgres",
    "password"  : "Ludi24"
}

def connect(params_dic):
    """ Connect to the PostgreSQL database server """
    conn = None
    try:
        # connect to the PostgreSQL server
        print('Connecting to the PostgreSQL database...')
        conn = psycopg2.connect(**params_dic)
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        sys.exit(1)
    print("Connection successful")
    return conn

connection = connect(param_dic)
connection

Connecting to the PostgreSQL database...
Connection successful


<connection object at 0x000002868FB0F378; dsn: 'user=postgres password=xxx dbname=cp2 host=localhost', closed: 0>

In [92]:
def insert_table(conn, df, table):
    """
    Using psycopg2.extras.insert_table() to insert the dataframe
    """
    # Create a list of tupples from the dataframe values
    tuples = [tuple(x) for x in df.to_numpy()]
    # Comma-separated dataframe columns
    cols = ','.join(list(df.columns))
    # SQL quert to execute
    query  = "INSERT INTO %s(%s) VALUES %%s" % (table, cols)
    cursor = conn.cursor()
    try:
        extras.execute_values(cursor, query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    print("insert_table() done")
    cursor.close

In [ ]:
insert_table(connection, df_volume[['id_enreg','name_volume','capacity','used_capacity', 'date', 'client','id_pool', 'id_typologie', 'id_origine']], 'stk_enregistrements')

In [113]:
# création d'un df contenant le nom des fichiers récupérés
df_fichiers_stockage_ibm = pd.DataFrame({'nom_fichier':fichiers_ibm_a_recuperer,'origine':np.array('ibm')})
df_fichiers_stockage_3par = pd.DataFrame({'nom_fichier':fichiers_3par_a_recuperer,'origine':np.array('3par')})
df_fichiers_stockage = pd.concat([df_fichiers_stockage_ibm,df_fichiers_stockage_3par])
df_fichiers_stockage = df_fichiers_stockage.merge(origines, on="origine")
df_fichiers_stockage

,nom_fichier,origine,id_origine
0,20210318_mutsapsan019_lssevdiskcopy.txt,ibm,0
1,20210317_mutapsan06_showvv-space.txt,3par,1


In [ ]:
insert_table(connection, df_fichiers_stockage[["nom_fichier","id_origine"]], 'stk_fichiers')